In [1]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [22]:
url = 'https://untappd.com/brewery/top_rated?country=united-states&brewery_type=regional_brewery'
driver = webdriver.Chrome(r'C:\Users\612456\Downloads\chromedriver_win32\chromedriver.exe')     #sets up chrome driver
driver.get(url)   #activates url (brings website up)

C:\Users\612456\AppData\Local\Temp\8\ipykernel_27944\1163545599.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\612456\Downloads\chromedriver_win32\chromedriver.exe')     #sets up chrome driver


In [25]:
# all these urls are breweries' beer lists
urls = ['https://untappd.com/VanishBeer/beer','https://untappd.com/TrilliumBrewing/beer', 'https://untappd.com/treehousebrewco/beer',
        'https://untappd.com/SaltyTurtleBeerCompany/beer', 'https://untappd.com/BearChaseBrewingCompany/beer','https://untappd.com/TexasLeaguer/beer',
        'https://untappd.com/WillowParkBrewing/beer','https://untappd.com/PipsMeadery/beer','https://untappd.com/EskerHartBrewingCompany/beer',
        'https://untappd.com/prolyfk/beer','https://untappd.com/Marlobobo/beer', 'https://untappd.com/EmperorsBrewery/beer',
        'https://untappd.com/SideProject/beer', 'https://untappd.com/troobado/beer', 'https://untappd.com/w/private-press-brewing/472291/beer', 
        'https://untappd.com/w/fidens-brewing-co/426930/beer', 'https://untappd.com/MindfulAles/beer', 'https://untappd.com/rootandbranchbrewing/beer'
        'https://untappd.com/WeaverHollowBrewery/beer', 'https://untappd.com/FreakFolkBier/beer', 'https://untappd.com/HorusAles/beer', 
        'https://untappd.com/CaseyBrewingBlending/beer', 'https://untappd.com/HouseofFermentology/beer', 'https://untappd.com/treehousebrewco/beer', 
        'https://untappd.com/barclaybrewingco/beer', 'https://untappd.com/Alchemist_Beer/beer', 'https://untappd.com/HillFarmsteadBrewery/beer',
        'https://untappd.com/monkishbrewing/beer','https://untappd.com/topplinggoliathbrewing/beer', 'https://untappd.com/OldNationBrewingCompany/beer',
        'https://untappd.com/PrairieArtisanAles/beer', 'https://untappd.com/fiddlehead/beer', 'https://untappd.com/NewEnglandBrewingCo/beer', 
        'https://untappd.com/fremontbrewing/beer', 'https://untappd.com/RevisionBrewingCompany/beer', 'https://untappd.com/JackieOsBrewery/beer',
        'https://untappd.com/westbrookbrewing/beer', 'https://untappd.com/creaturecomforts/beer', 'https://untappd.com/LaCumbreBrewingCompany/beer',
        'https://untappd.com/SurlyBrewingCompany/beer','https://untappd.com/pfriembeer/beer','https://untappd.com/RevolutionBrewingChicago/beer',
        'https://untappd.com/NewGlarusBrewingCompany/beer','https://untappd.com/halfacrebeer/beer','https://untappd.com/TheStarkeller/beer',
        'https://untappd.com/HenHouseBrewing/beer','https://untappd.com/MelvinBrewing/beer','https://untappd.com/AlpineBeerCo/beer',
        'https://untappd.com/Rhinegeist/beer','https://untappd.com/hardywood/beer', 'https://untappd.com/portcitybrewing/beer'] 

In [26]:
print('Number of different Breweries: ',len(urls))
#I accidenty added a meadery, that can either be good or bad tbh but just noting tgis so I don't forget it

Number of different Breweries:  50


In [27]:
name_ls = []
style_ls = []
desc_ls = []
abv_ls = []
ibu_ls = []

In [28]:
for i in range(len(urls)):   #goes through all the urls/breweries
    url=urls[i]    #sets url to new brewery
    driver = webdriver.Chrome(r'C:\Users\612456\Downloads\chromedriver_win32\chromedriver.exe')     #sets up chrome driver
    driver.get(url)   #activates url (brings website up)
    beers = driver.find_elements_by_css_selector('div.beer-item')     #finds each individual beer and sets it to a "list" of beers
    
    #this loop goes through all the beers in the list and clicks "show more" to get the full description
    for i in range(len(beers)):
        beer_det = beers[i].find_element_by_css_selector('div.beer-details')
        beer_det.location_once_scrolled_into_view
        try:
            beer_det.find_element_by_css_selector('a.read-more-beerlist.track-click').click()
        except NoSuchElementException:
            continue
        beer_det.location_once_scrolled_into_view
    for beer in beers:   #goes through each beer in the "list" and gets the name, style, IBU, abv, and description (half)
        
        try:
            name = beer.find_element_by_css_selector('p.name').text    #extracts name of beer
            style = beer.find_element_by_css_selector('p.style').text   #extracts style of beer
            abv = beer.find_element_by_css_selector('div.details-item.abv').text   #extracts abv of beer
            ibu = beer.find_element_by_css_selector('div.details-item.ibu').text   #extracts ibu of beer
            desc = beer.find_element_by_xpath('.//*[@class="beer-details"]/p[4]').text[:-11]   #gets the full description and also cuts off the 'show more'
            if desc=='': #if desc isn't long enough to have a "read More" then get the other desc which is stored where the half one is
                desc = beer.find_element_by_xpath('.//*[@class="beer-details"]/p[3]').text
                
        except NoSuchElementException:
            abv = beer.find_element_by_css_selector('p.abv').text   # part of the way through the process the namespace 
            ibu = beer.find_element_by_css_selector('p.ibu').text   # changes for ibu and abv so this handles that
    
        name_ls.append(name)     #these add all the attributes to a list that will eventually be set to a df
        style_ls.append(style)
        desc_ls.append(desc)    
        abv_ls.append(abv)             #(I have not seen a description too short to not hae one but that could be a potential danger)
        ibu_ls.append(ibu)
        
        print("Name: ", name,'\nStyle:',style,'\nABV: ',abv,'\nIBU: ',ibu,'\nDescription: ',desc,'\n\n------------\n')

C:\Users\612456\AppData\Local\Temp\8\ipykernel_27944\789644729.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\612456\Downloads\chromedriver_win32\chromedriver.exe')     #sets up chrome driver
C:\Users\612456\AppData\Local\Temp\8\ipykernel_27944\789644729.py:5: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  beers = driver.find_elements_by_css_selector('div.beer-item')     #finds each individual beer and sets it to a "list" of beers
C:\Users\612456\AppData\Local\Programs\Python\Python310\lib\site-packages\selenium\webdriver\remote\webelement.py:484: UserWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  warnings.warn("find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead")
C:

Name:  Ghost Fleet 
Style: IPA - American 
ABV:  7.1% ABV 
IBU:  N/A IBU 
Description:  This is our house IPA. It is hugely hoppy with fruity aromatics from Amarillo and Simcoe hops. A hazy and unfiltered beer with a smooth finish. 

------------

Name:  Fat Boys 
Style: IPA - Imperial / Double 
ABV:  8.5% ABV 
IBU:  N/A IBU 
Description:  This is our Double IPA. It features bold fruity aromatics commonly associate with American IPAs. It contains a ton of hops including Citra and Equinox. As far as DIPAs go this one is light-bodied with an easy finish 

------------

Name:  Wraith 
Style: Wheat Beer - Witbier / Blanche 
ABV:  5.3% ABV 
IBU:  N/A IBU 
Description:  Wraith is a Belgian Style Witbier that provides you with notes of citrus, chamomile and coriander. 

------------

Name:  Lucketts Calling 
Style: IPA - American 
ABV:  6.3% ABV 
IBU:  N/A IBU 
Description:  A hazy IPA with big fruity hop flavors and almost no bitterness. 

------------

Name:  Beyond The Farm 
Style: IPA - A

KeyboardInterrupt: 

1214

#### the above cell takes ~40 mins to run

### Data is recorded in the lists now add to df

In [311]:
#create df
df = pd.DataFrame()
df['Name'] = name_ls
df['style'] = style_ls
df['IBU'] = ibu_ls
df['ABV'] = abv_ls
df['Description'] = desc_ls
df

,Name,style,IBU,ABV,Description
0,Ghost Fleet,IPA - American,N/A IBU,7.1% ABV,This is our house IPA. It is hugely hoppy with...
1,Fat Boys,IPA - Imperial / Double,N/A IBU,8.5% ABV,This is our Double IPA. It features bold fruit...
2,Wraith,Wheat Beer - Witbier / Blanche,N/A IBU,5.3% ABV,Wraith is a Belgian Style Witbier that provide...
3,Lucketts Calling,IPA - American,N/A IBU,6.3% ABV,A hazy IPA with big fruity hop flavors and alm...
4,Beyond The Farm,IPA - American,N/A IBU,6.8% ABV,"Relying on the citrusy Summit, Citra, and Cent..."
...,...,...,...,...,...
1209,Ideaal Tripel,Belgian Tripel,35 IBU,8.5% ABV,Drawing inspiration from Belgium’s monastic br...
1210,Hazy ipa,IPA - New England / Hazy,35 IBU,6.8% ABV,"Brewed to highlight hop aroma and flavor, Hazy..."
1211,Star Sailor®,IPA - White,40 IBU,6.2% ABV,"Exploring the limits of IPAs, Star Sailor® Whi..."
1212,German Pilsner,Pilsner - German,35 IBU,4.6% ABV,"Looking to Germany for inspiration, our German..."


In [312]:
#get rid of all beers that don't have all 3 major things I need
df = df[(df['IBU']!='N/A IBU')&(df['ABV']!='N/A ABV')&(df['Description']!='')]

In [325]:
df

,Name,style,IBU,ABV,Description
26,Vicinity,IPA - Imperial / Double,65,8,Originally brewed in celebration of the first ...
33,Galaxy Dry Hopped Fort Point,Pale Ale - American,45,6.6,This version of our signature American pale al...
47,Launch Beer,Pale Ale - American,68,5.7,First brewed to commemorate the grand opening ...
50,Julius,IPA - American,72,6.8,"Bursting with pungent American hops, Julius - ..."
51,Haze,IPA - Imperial / Double,90,8.2,Our Double IPA! We smell a ton of peach on the...
...,...,...,...,...,...
1208,Mexican Dark Lager,Lager - Vienna,24,5,Port City’s Mexican Dark Lager draws inspirati...
1209,Ideaal Tripel,Belgian Tripel,35,8.5,Drawing inspiration from Belgium’s monastic br...
1210,Hazy ipa,IPA - New England / Hazy,35,6.8,"Brewed to highlight hop aroma and flavor, Hazy..."
1211,Star Sailor®,IPA - White,40,6.2,"Exploring the limits of IPAs, Star Sailor® Whi..."


In [324]:
df['IBU'] = df['IBU'].str[:-4]

C:\Users\612456\AppData\Local\Temp\7\ipykernel_18756\849542697.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IBU'] = df['IBU'].str[:-4]


In [321]:
df['ABV'] = df['ABV'].str[:-5]

C:\Users\612456\AppData\Local\Temp\7\ipykernel_18756\2652638095.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ABV'] = df['ABV'].str[:-5]


In [326]:
df['ABV'] = df['ABV'].astype(float)

C:\Users\612456\AppData\Local\Temp\7\ipykernel_18756\2667934893.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ABV'] = df['ABV'].astype(float)


In [327]:
df['IBU'] = df['IBU'].astype(int)

C:\Users\612456\AppData\Local\Temp\7\ipykernel_18756\3279095354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IBU'] = df['IBU'].astype(int)


In [328]:
df

,Name,style,IBU,ABV,Description
26,Vicinity,IPA - Imperial / Double,65,8.0,Originally brewed in celebration of the first ...
33,Galaxy Dry Hopped Fort Point,Pale Ale - American,45,6.6,This version of our signature American pale al...
47,Launch Beer,Pale Ale - American,68,5.7,First brewed to commemorate the grand opening ...
50,Julius,IPA - American,72,6.8,"Bursting with pungent American hops, Julius - ..."
51,Haze,IPA - Imperial / Double,90,8.2,Our Double IPA! We smell a ton of peach on the...
...,...,...,...,...,...
1208,Mexican Dark Lager,Lager - Vienna,24,5.0,Port City’s Mexican Dark Lager draws inspirati...
1209,Ideaal Tripel,Belgian Tripel,35,8.5,Drawing inspiration from Belgium’s monastic br...
1210,Hazy ipa,IPA - New England / Hazy,35,6.8,"Brewed to highlight hop aroma and flavor, Hazy..."
1211,Star Sailor®,IPA - White,40,6.2,"Exploring the limits of IPAs, Star Sailor® Whi..."


In [16]:
# save the df to a csv that I can use later and don't have to run this every time
df.to_csv('untappd_beers.csv')

#### Skip the cells before and run this cell if you already have the data you want

In [2]:
df = pd.read_csv('untappd_beers.csv',encoding='latin-1')
df.drop('Unnamed: 0', axis=1, inplace=True)

In [3]:
df

,Name,style,IBU,ABV,Description
0,Vicinity,IPA - Imperial / Double,65,8.0,Originally brewed in celebration of the first ...
1,Galaxy Dry Hopped Fort Point,Pale Ale - American,45,6.6,This version of our signature American pale al...
2,Launch Beer,Pale Ale - American,68,5.7,First brewed to commemorate the grand opening ...
3,Julius,IPA - American,72,6.8,"Bursting with pungent American hops, Julius - ..."
4,Haze,IPA - Imperial / Double,90,8.2,Our Double IPA! We smell a ton of peach on the...
...,...,...,...,...,...
536,Mexican Dark Lager,Lager - Vienna,24,5.0,Port Cityâs Mexican Dark Lager draws inspira...
537,Ideaal Tripel,Belgian Tripel,35,8.5,Drawing inspiration from Belgiumâs monastic ...
538,Hazy ipa,IPA - New England / Hazy,35,6.8,"Brewed to highlight hop aroma and flavor, Hazy..."
539,Star SailorÂ®,IPA - White,40,6.2,"Exploring the limits of IPAs, Star SailorÂ® Wh..."


In [4]:
styles = df['style'].unique().tolist()

In [17]:
styles

['IPA - Imperial / Double',
 'Pale Ale - American',
 'IPA - American',
 'Stout - Milk / Sweet',
 'IPA - New England / Hazy',
 'Stout - Oatmeal',
 'MÃ¤rzen',
 'Bock - Single / Traditional',
 'Pumpkin / Yam Beer',
 'Pale Ale - Belgian',
 'Blonde Ale',
 'Pilsner - Czech',
 'Stout - Imperial / Double Oatmeal',
 'IPA - Triple New England / Hazy',
 'Stout - Imperial / Double',
 'Barleywine - English',
 'Stout - Imperial / Double Milk',
 'Porter - Imperial / Double',
 'Sour - Berliner Weisse',
 'Belgian Quadrupel',
 'Barleywine - American',
 'Stout - Russian Imperial',
 'Barleywine - Other',
 'Stout - Foreign / Export',
 'Farmhouse Ale - Saison',
 'Lager - Helles',
 'Stout - Imperial / Double Coffee',
 'Stout - Imperial / Double Pastry',
 'Stout - Other',
 'Wild Ale - American',
 'Sour - Other',
 'Wild Ale - Other',
 'IPA - Imperial / Double New England / Hazy',
 'Sour - Flanders Oud Bruin',
 'Wheat Beer - Witbier / Blanche',
 'Pilsner - German',
 'Belgian Tripel',
 'Pale Ale - New England / 

In [6]:
sour_ls = [df['style'].iloc[i] for i in range(len(df)) if 'Sour' in df['style'].iloc[i]]

In [76]:
style_df = df[df['style'].str.contains('Honey')]
style_df.shape

(1, 6)

In [ ]:
# Pale Ale 40
# IPA 218
# Stout 92
# Sour 43
# Lager 9 ,Pilsner 12, Bock 2, 'MÃ¤rzen' 4
# Bock 2
# Blonde Ale 2
# 'MÃ¤rzen' 4
# Brown Ale 7
# Wheat 11
# Wild Ale 20
# Red Ale 4

In [ ]:
###### Styles i Think i might try to do here
# IPA 218
# Stout 92
# Wild Ale [Sour - Fruited Gose, Sour - Fruited, Sour - Berlinner Weisse] 63
# Pale Ale 40
# lager [Pilsner, Bock, Marzen] 27
# Belgian ['Belgian Pale Ale','Belgian Strong Pale Ale','Belgian Dark Ale','Saison / Farmhouse Ale', 'Tripel','Dubbel'] 15
# Porter 11
# Wheat Beer 11

#need to get more of everything especially wheat, porter, belgian, lager, pale ale


In [225]:
92 + 63+40+27

222

In [55]:
# so I could start off by seeing if it can identify an IPA and compare the IPAs to 
 # Stout, Sours, Pale Ale, and Lager, and mayyybe Belgian

In [60]:
#creates binary column 'is_IPA'
df['is_IPA'] = [0]*len(df)
for i in range(len(df)):
    if 'IPA' in df['style'].iloc[i]:
        df['is_IPA'].iloc[i]=1

C:\Users\612456\AppData\Local\Temp\1\ipykernel_11096\2248898157.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_IPA'].iloc[i]=1


In [176]:
# gets the descriptions to be all lower case
for i in range(len(df)):
    df['Description'].iloc[i] = df['Description'].iloc[i].lower()

C:\Users\612456\AppData\Local\Temp\1\ipykernel_11096\2198410040.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Description'].iloc[i] = df['Description'].iloc[i].lower()


In [177]:
#words that I need to remove from the descriptions of the beers because they will give the style away
scratch_words = ['ipa', 'hazy', 'west coast', 'belgian', 'porter', 'pale', 'ale', 'wild', 'sour', 'stout', 
                 'lager', 'saison', 'farmhouse', 'dubbel', 'tripel', 'mã¤rzen', 'tree house family', 'american',
                 'india', 'blonde']
# might keep 'hazy', but could be too obvious

In [215]:
tst_styles = ['IPA', 'Stout', 'Wild', 'Sour', 'Pale Ale', 'Lager', 'Pilsner', 'Bock', 'MÃ¤rzen']
test_df = df[(df['style'].str.contains('|'.join(tst_styles)))]   #438 beers

In [217]:
#goes through all the rows of the df and removes the scratch_words
for i in range(len(test_df)):
    tmp_str = test_df['Description'].iloc[i].split()
    resultwords = [word for word in tmp_str if word not in scratch_words]
    test_df['Description'].iloc[i] = ' '.join(resultwords)

C:\Users\612456\AppData\Local\Temp\1\ipykernel_11096\4093491772.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Description'].iloc[i] = ' '.join(resultwords)


In [218]:
#try removing stopwords and stemming the words
from nltk.corpus import stopwords
import nltk.data
import nltk

nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\612456\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [220]:
#removes stopwords, stems words, and removes anything not an alpha ** and numeric?? char
from nltk.stem.porter import PorterStemmer

for i in range(len(test_df)):
    tmp_str = test_df['Description'].iloc[i].split()
    tmp_str = [word for word in tmp_str if not word in stop_words]
    tmp_str = [word for word in tmp_str if word.isalpha()]
    tmp_str = [PorterStemmer().stem(word) for word in tmp_str]
    test_df['Description'].iloc[i] = ' '.join(tmp_str)

C:\Users\612456\AppData\Local\Temp\1\ipykernel_11096\39512009.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Description'].iloc[i] = ' '.join(tmp_str)


In [221]:
#testing a ml model for determining if it is an IPA or not
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

X = test_df['Description']
y = test_df['is_IPA']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [222]:
vect = CountVectorizer()
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

In [223]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)
print('accuracy = ',(metrics.accuracy_score(y_test,y_pred_class)))

accuracy =  0.8818181818181818


In [224]:
print('Percent IPA:', y_test.mean())
print('Percent non IPA:', 1 - y_test.mean())

Percent IPA: 0.4818181818181818
Percent non IPA: 0.5181818181818182


## Runnig Results
    - Baseline of 48%
    - Inital test accuracy was 81%
    - next I removed stopwords and stemmed the words to get an accuracy of 88%